# Income Prediction Using Decision Tree Regression

## Loading The Main Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

## Data Collection

In [2]:
data = pd.read_csv('/content/data.csv')

In [3]:
data.head()

,Age,Education_Level,Occupation,Number_of_Dependents,Location,Work_Experience,Marital_Status,Employment_Status,Household_Size,Homeownership_Status,Type_of_Housing,Gender,Primary_Mode_of_Transportation,Income
0,56,Master's,Technology,5,Urban,21,Married,Full-time,7,Own,Apartment,Male,Public transit,72510
1,69,High School,Finance,0,Urban,4,Single,Full-time,7,Own,Apartment,Male,Biking,75462
2,46,Bachelor's,Technology,1,Urban,1,Single,Full-time,7,Own,Single-family home,Female,Car,71748
3,32,High School,Others,2,Urban,32,Married,Full-time,1,Own,Apartment,Female,Car,74520
4,60,Bachelor's,Finance,3,Urban,15,Married,Self-employed,4,Own,Townhouse,Male,Walking,640210


## EDA

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Age                             10000 non-null  int64 
 1   Education_Level                 10000 non-null  object
 2   Occupation                      10000 non-null  object
 3   Number_of_Dependents            10000 non-null  int64 
 4   Location                        10000 non-null  object
 5   Work_Experience                 10000 non-null  int64 
 6   Marital_Status                  10000 non-null  object
 7   Employment_Status               10000 non-null  object
 8   Household_Size                  10000 non-null  int64 
 9   Homeownership_Status            10000 non-null  object
 10  Type_of_Housing                 10000 non-null  object
 11  Gender                          10000 non-null  object
 12  Primary_Mode_of_Transportation  10000 non-null 

In [5]:
data.describe()

,Age,Number_of_Dependents,Work_Experience,Household_Size,Income
count,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04
mean,44.021700,2.527000,24.858800,3.989600,8.168382e+05
std,15.203998,1.713991,14.652622,2.010496,1.821089e+06
min,18.000000,0.000000,0.000000,1.000000,3.104400e+04
25%,31.000000,1.000000,12.000000,2.000000,6.844600e+04
50%,44.000000,3.000000,25.000000,4.000000,7.294300e+04
75%,57.000000,4.000000,37.000000,6.000000,3.506675e+05
max,70.000000,5.000000,50.000000,7.000000,9.992571e+06


In [6]:
data.isna().sum()

,0
Age,0
Education_Level,0
Occupation,0
Number_of_Dependents,0
Location,0
Work_Experience,0
Marital_Status,0
Employment_Status,0
Household_Size,0
Homeownership_Status,0


In [7]:
data.nunique()

,0
Age,53
Education_Level,4
Occupation,5
Number_of_Dependents,6
Location,3
Work_Experience,51
Marital_Status,3
Employment_Status,3
Household_Size,7
Homeownership_Status,2


In [8]:
for col in data.drop('Income' , axis = 1).columns:
  print(data[col].value_counts())

Age
43    223
66    216
62    215
40    212
64    209
34    209
52    207
45    205
38    203
35    200
49    200
30    200
46    199
42    197
19    196
29    194
21    194
39    194
56    192
68    192
57    191
59    191
32    191
54    191
37    190
41    189
51    189
22    188
61    188
25    188
20    188
33    187
53    186
23    185
36    183
47    183
55    183
28    182
70    182
50    182
69    178
26    177
18    176
65    175
31    175
67    174
27    174
58    173
63    169
60    162
24    161
44    159
48    153
Name: count, dtype: int64
Education_Level
Bachelor's     4058
High School    2959
Master's       2482
Doctorate       501
Name: count, dtype: int64
Occupation
Healthcare    3035
Technology    2407
Finance       1525
Others        1521
Education     1512
Name: count, dtype: int64
Number_of_Dependents
5    1745
3    1712
1    1651
0    1642
4    1629
2    1621
Name: count, dtype: int64
Location
Urban       7037
Suburban    1951
Rural       1012
Name: count, dtype:

In [9]:
fig_income_hist = px.histogram(data, x='Income', title='Distribution of Income')
fig_income_hist.show()

## Data Preprocessing

### Handling the missing values

      - No missing values

### Encoding the categorical variables

In [10]:
categorical_cols = data.select_dtypes( include = 'object' ).columns

display ( categorical_cols )

Index(['Education_Level', 'Occupation', 'Location', 'Marital_Status',
       'Employment_Status', 'Homeownership_Status', 'Type_of_Housing',
       'Gender', 'Primary_Mode_of_Transportation'],
      dtype='object')

      ▶ categorical_cols = [ Occupation  ,  Location  ,  Marital_Status  , Employment_Status  ,  Homeownership_Status  ,

      Type_of_Housing  ,  Gender  ,  Primary_Mode_of_Transportation  ,  Education_Level  ]
      

      ▶ Encode the Education_Level using Ordinal Encoding  because it's an ordinal variable


      ▶ Encode the reminder Using OneHot Encoding method

      ▶  First , We will Encode the Education_Level using Ordinal Encoding because it's an ordinal variable

In [11]:
data['Education_Level'].value_counts()

,count
Education_Level,
Bachelor's,4058
High School,2959
Master's,2482
Doctorate,501


In [12]:
data['Education_Level'].head(10)

,Education_Level
0,Master's
1,High School
2,Bachelor's
3,High School
4,Bachelor's
5,High School
6,Master's
7,Master's
8,Master's
9,Bachelor's


In [13]:
data['Education_Level'].tail(10)

,Education_Level
9990,Bachelor's
9991,Bachelor's
9992,Master's
9993,Bachelor's
9994,High School
9995,High School
9996,Master's
9997,Doctorate
9998,High School
9999,High School


In [14]:
# Encode the Education_Level using Ordinal Encoding method because it's an ordinal variable

EducationLevelMap = {
    "High School": 0,
    "Bachelor's": 1,
    "Master's": 2,
    "Doctorate": 3
}

data['Education_Level'] = data['Education_Level'].map( EducationLevelMap )

In [15]:
data['Education_Level'].head(10)

,Education_Level
0,2
1,0
2,1
3,0
4,1
5,0
6,2
7,2
8,2
9,1


In [16]:
data['Education_Level'].tail(10)

,Education_Level
9990,1
9991,1
9992,2
9993,1
9994,0
9995,0
9996,2
9997,3
9998,0
9999,0


      ▶ Encoding the reminder categorical variables using OneHot Encoding

      ▶ No need to drop any dummy columns after getting the dummy columns from the OneHot because the Decesion Tree Regression

       isn't affected by the multicollinearity

In [17]:
categorical_cols = data.select_dtypes( include = 'object' ).columns

#categorical_cols

In [18]:
data = pd.get_dummies( data , columns = categorical_cols , dtype = int )      # OneHot Encoding for the reminder categorical columns

In [19]:
data.head()

,Age,Education_Level,Number_of_Dependents,Work_Experience,Household_Size,Income,Occupation_Education,Occupation_Finance,Occupation_Healthcare,Occupation_Others,...,Homeownership_Status_Rent,Type_of_Housing_Apartment,Type_of_Housing_Single-family home,Type_of_Housing_Townhouse,Gender_Female,Gender_Male,Primary_Mode_of_Transportation_Biking,Primary_Mode_of_Transportation_Car,Primary_Mode_of_Transportation_Public transit,Primary_Mode_of_Transportation_Walking
0,56,2,5,21,7,72510,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
1,69,0,0,4,7,75462,0,1,0,0,...,0,1,0,0,0,1,1,0,0,0
2,46,1,1,1,7,71748,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
3,32,0,2,32,1,74520,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
4,60,1,3,15,4,640210,0,1,0,0,...,0,0,0,1,0,1,0,0,0,1


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 31 columns):
 #   Column                                         Non-Null Count  Dtype
---  ------                                         --------------  -----
 0   Age                                            10000 non-null  int64
 1   Education_Level                                10000 non-null  int64
 2   Number_of_Dependents                           10000 non-null  int64
 3   Work_Experience                                10000 non-null  int64
 4   Household_Size                                 10000 non-null  int64
 5   Income                                         10000 non-null  int64
 6   Occupation_Education                           10000 non-null  int64
 7   Occupation_Finance                             10000 non-null  int64
 8   Occupation_Healthcare                          10000 non-null  int64
 9   Occupation_Others                              10000 non-null  int64
 10 

.

### Splitting the Data into input & output

In [21]:
x = data.drop('Income' , axis = 1)
y = data['Income']

In [22]:
x.head()

,Age,Education_Level,Number_of_Dependents,Work_Experience,Household_Size,Occupation_Education,Occupation_Finance,Occupation_Healthcare,Occupation_Others,Occupation_Technology,...,Homeownership_Status_Rent,Type_of_Housing_Apartment,Type_of_Housing_Single-family home,Type_of_Housing_Townhouse,Gender_Female,Gender_Male,Primary_Mode_of_Transportation_Biking,Primary_Mode_of_Transportation_Car,Primary_Mode_of_Transportation_Public transit,Primary_Mode_of_Transportation_Walking
0,56,2,5,21,7,0,0,0,0,1,...,0,1,0,0,0,1,0,0,1,0
1,69,0,0,4,7,0,1,0,0,0,...,0,1,0,0,0,1,1,0,0,0
2,46,1,1,1,7,0,0,0,0,1,...,0,0,1,0,1,0,0,1,0,0
3,32,0,2,32,1,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
4,60,1,3,15,4,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1


In [23]:
y.head()

,Income
0,72510
1,75462
2,71748
3,74520
4,640210


.

### Splitting the data into training and testing

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
x_train , x_test , y_train , y_test = train_test_split ( x , y , test_size = 0.2 , random_state = 0 )

In [26]:
x_train.head()

,Age,Education_Level,Number_of_Dependents,Work_Experience,Household_Size,Occupation_Education,Occupation_Finance,Occupation_Healthcare,Occupation_Others,Occupation_Technology,...,Homeownership_Status_Rent,Type_of_Housing_Apartment,Type_of_Housing_Single-family home,Type_of_Housing_Townhouse,Gender_Female,Gender_Male,Primary_Mode_of_Transportation_Biking,Primary_Mode_of_Transportation_Car,Primary_Mode_of_Transportation_Public transit,Primary_Mode_of_Transportation_Walking
7389,42,0,1,10,2,0,0,0,1,0,...,1,1,0,0,0,1,1,0,0,0
9275,19,1,0,21,2,0,0,1,0,0,...,1,1,0,0,0,1,0,0,1,0
2995,49,2,3,10,4,0,0,0,1,0,...,1,0,0,1,0,1,0,0,1,0
5316,68,2,4,5,5,0,0,0,1,0,...,1,0,1,0,0,1,1,0,0,0
356,39,1,4,37,6,0,0,0,1,0,...,0,1,0,0,1,0,1,0,0,0


In [27]:
y_train.head()

,Income
7389,71692
9275,279059
2995,280479
5316,2280108
356,2771585


In [28]:
x_test.head()

,Age,Education_Level,Number_of_Dependents,Work_Experience,Household_Size,Occupation_Education,Occupation_Finance,Occupation_Healthcare,Occupation_Others,Occupation_Technology,...,Homeownership_Status_Rent,Type_of_Housing_Apartment,Type_of_Housing_Single-family home,Type_of_Housing_Townhouse,Gender_Female,Gender_Male,Primary_Mode_of_Transportation_Biking,Primary_Mode_of_Transportation_Car,Primary_Mode_of_Transportation_Public transit,Primary_Mode_of_Transportation_Walking
9394,69,0,3,34,5,1,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0
898,31,1,1,47,2,0,1,0,0,0,...,0,1,0,0,1,0,0,1,0,0
2398,19,0,0,42,3,0,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0
5906,55,2,4,29,6,0,0,1,0,0,...,0,0,0,1,0,1,1,0,0,0
2343,67,0,0,49,1,1,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0


In [29]:
y_test.head()

,Income
9394,74532
898,65380
2398,67508
5906,67726
2343,67574


In [30]:
x_train = x_train.values
y_train = y_train.values
x_test  = x_test.values
y_test  = y_test.values

In [31]:
x_train

array([[42,  0,  1, ...,  0,  0,  0],
       [19,  1,  0, ...,  0,  1,  0],
       [49,  2,  3, ...,  0,  1,  0],
       ...,
       [54,  1,  4, ...,  0,  1,  0],
       [37,  1,  1, ...,  0,  1,  0],
       [26,  1,  3, ...,  0,  0,  1]])

### Log Transformation

      👉 Our Decision Tree Model is highly affected by the outliers❗ , because it depends on the mean✅ at it's predictions
      
      results let we have a target values in the leaf node after finishing splitting like [ 10 , 15 , 1000 ] , the mean will  

      equals 341.6 so if i with the same x values which lead to this leaf node tring to predict a value it will always give me the
      
      result = 341.6 , and it's a very bad prediction because this model is highly affected by the outliers


      👉 Also this model is highly affected by the high variances between the target values


      👉 How to handle the variance and outliers problem ?

      ▶ Using Log Transformation : np.log1p ✅

      Here it will apply loge( y+1 ) ✅ to each value in y so that the whole values will be in the same range

      variance will be very small , and the outliers will be compressed to many in the normal range as example ▶
      
       الدخل بعد التحويل     ,                 الدخل الاصلى (Log Scale)
          72,510                 ,        11.19
          75,462                 ,        11.23
          2,771,585 (Outlier)    ,        14.83

      Look now the whole values became in the same range so that now we overcommed the mean and variance problem
      
      so that our model now will not affected by these high values and the y distribution will converted into a

      normal distribution ✅ instead of the skewed dastribution ❎

      ▶ After training our model on the log scaled y we will predict the y values using np.expm1✅ which is opposite of
      
      the loge( y+1 )


      ▶ Instead of the log transormations we can drop the outliers using the interquartile range IQR = Q3 - Q1 then drop any

      values not in the range [ Q1 - 1.5 * IQR , Q3 + 1.5 * IQR ] but now we will drop from [ 1.5% to 5% of the rows ]



      ✅ : The best option is the log tranformation

      ✅ : It's preferrable to apply the log tranformation after the split not before to keep the pipling processes ( مسار العمل )

      in ML , Although no Data Leakage ❌ happens in log transformation but it's preferrable to keep the pipling in ML projects .

      ✅ pipling in ML projects : Split then ▶ Transform

In [32]:
yTrainLog = np.log1p( y_train )
yTestLog  = np.log1p( y_test  )

In [33]:
yTrainLog

array([11.18014839, 12.53918209, 12.5442577 , ..., 11.22453648,
       11.20714731, 10.92401217])

In [34]:
yTestLog

array([11.21899726, 11.08798698, 11.1200162 , ..., 11.1260122 ,
       10.4453757 , 10.91703203])

## Feature Selection

      ▶ Decision Tree Regression is not affected by the multicollinearity so even if there is a non important features it will not

      split from it so it will be ignored

## Model Training

In [35]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [36]:
dtree = DecisionTreeRegressor( random_state = 0 )

In [37]:
param_grid = {
    'max_depth': [ 2, 3 , 4, 5 , 6 , 7 , 8, 10, 12, 14, 16 , None ],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8],
    'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]
}

In [38]:
getBestTree = GridSearchCV( estimator = dtree , param_grid = param_grid , cv = 5 , scoring = 'neg_mean_squared_error' , n_jobs = -1 , verbose = 2 )

getBestTree.fit( x_train , yTrainLog )

print("GridSearchCV completed successfully and best hyperparameters have been identified.")
print("Best Found Max Depth From GridSearchCV Is:", getBestTree.best_params_)
print("Best Average CV score:", -1 * getBestTree.best_score_ , '\n' )


Fitting 5 folds for each of 864 candidates, totalling 4320 fits
GridSearchCV completed successfully and best hyperparameters have been identified.
Best Found Max Depth From GridSearchCV Is: {'max_depth': 7, 'min_samples_leaf': 8, 'min_samples_split': 2}
Best Average CV score: 2.1557218284536677 



In [39]:
regressor =  getBestTree.best_estimator_                # our final best trained model

## Model Testing

In [40]:
predictedIncomeLog = regressor.predict( x_test )

In [41]:
RealpredictedIncome = np.expm1( predictedIncomeLog )

### Model Evaluation

In [42]:
from sklearn.metrics import mean_squared_error , r2_score

### Evaluate Training results

In [50]:
LogTrainingPrediction  = regressor.predict( x_train )
RealTrainingPrediction = np.expm1( LogTrainingPrediction )

TrainingResults = pd.DataFrame( x_train, columns= x.columns )
TrainingResults['True_Income'] = y_train
TrainingResults['Predicted_Income'] =  np.round( RealTrainingPrediction )

TrainingResults.head( 10 )

,Age,Education_Level,Number_of_Dependents,Work_Experience,Household_Size,Occupation_Education,Occupation_Finance,Occupation_Healthcare,Occupation_Others,Occupation_Technology,...,Type_of_Housing_Single-family home,Type_of_Housing_Townhouse,Gender_Female,Gender_Male,Primary_Mode_of_Transportation_Biking,Primary_Mode_of_Transportation_Car,Primary_Mode_of_Transportation_Public transit,Primary_Mode_of_Transportation_Walking,True_Income,Predicted_Income
0,42,0,1,10,2,0,0,0,1,0,...,0,0,0,1,1,0,0,0,71692,202521.0
1,19,1,0,21,2,0,0,1,0,0,...,0,0,0,1,0,0,1,0,279059,336427.0
2,49,2,3,10,4,0,0,0,1,0,...,0,1,0,1,0,0,1,0,280479,202521.0
3,68,2,4,5,5,0,0,0,1,0,...,1,0,0,1,1,0,0,0,2280108,202521.0
4,39,1,4,37,6,0,0,0,1,0,...,0,0,1,0,1,0,0,0,2771585,253830.0
5,35,1,5,20,1,0,0,0,0,1,...,0,0,0,1,1,0,0,0,590478,206518.0
6,24,3,5,27,4,0,0,1,0,0,...,1,0,1,0,1,0,0,0,70160,69734.0
7,62,1,1,49,6,0,0,0,0,1,...,1,0,1,0,0,1,0,0,66054,75478.0
8,61,0,0,15,7,0,0,0,1,0,...,0,0,1,0,0,0,0,1,8924221,198029.0
9,70,2,2,15,3,0,0,1,0,0,...,0,0,1,0,0,0,1,0,105979,123400.0


In [52]:
TrainingMse = mean_squared_error( y_train , RealTrainingPrediction )
TrainingR2  = r2_score( y_train , RealTrainingPrediction )

print(f"Training Mean Squared Error: {TrainingMse:.2f}")
print(f"Training R-squared: {TrainingR2*100:.2f}%")

Training Mean Squared Error: 3544225330103.56
Training R-squared: -5.97%


### Evaluating the test results

In [44]:
TestingResults = pd.DataFrame( x_test, columns= x.columns )
TestingResults ['True_Income'] = y_test
TestingResults ['Predicted_Income'] =  np.round( RealpredictedIncome )

TestingResults.head( 10 )

,Age,Education_Level,Number_of_Dependents,Work_Experience,Household_Size,Occupation_Education,Occupation_Finance,Occupation_Healthcare,Occupation_Others,Occupation_Technology,...,Type_of_Housing_Single-family home,Type_of_Housing_Townhouse,Gender_Female,Gender_Male,Primary_Mode_of_Transportation_Biking,Primary_Mode_of_Transportation_Car,Primary_Mode_of_Transportation_Public transit,Primary_Mode_of_Transportation_Walking,True_Income,Predicted_Income
0,69,0,3,34,5,1,0,0,0,0,...,0,1,0,1,0,1,0,0,74532,103309.0
1,31,1,1,47,2,0,1,0,0,0,...,0,0,1,0,0,1,0,0,65380,331883.0
2,19,0,0,42,3,0,0,1,0,0,...,0,0,0,1,0,1,0,0,67508,253830.0
3,55,2,4,29,6,0,0,1,0,0,...,0,1,0,1,1,0,0,0,67726,157868.0
4,67,0,0,49,1,1,0,0,0,0,...,1,0,1,0,1,0,0,0,67574,75478.0
5,38,1,1,16,7,0,0,0,0,1,...,1,0,1,0,0,1,0,0,73802,103537.0
6,47,1,2,37,3,0,0,0,1,0,...,1,0,0,1,0,1,0,0,490746,147811.0
7,31,2,3,10,6,0,0,0,1,0,...,0,0,0,1,1,0,0,0,3226293,287147.0
8,40,1,5,2,4,0,1,0,0,0,...,1,0,0,1,0,1,0,0,67954,68988.0
9,18,0,4,1,6,0,0,0,1,0,...,0,0,0,1,0,0,0,1,68228,287147.0


In [45]:
TestingMse = mean_squared_error( y_test , RealpredictedIncome )
TestingR2  = r2_score( y_test , RealpredictedIncome )

print(f"Testing Mean Squared Error: {TestingMse:.2f}")
print(f"Testing R-squared: {TestingR2*100:.2f}%")

Testing Mean Squared Error: 3474721288864.48
Testing R-squared: -8.54%


.

## Visualizing the results

In [46]:
import plotly.express as px

In [47]:
fig_train = px.scatter(
    TrainingResults,
    x='True_Income',
    y='Predicted_Income',
    title='True vs. Predicted Income (Training Data)',
    labels={'True_Income': 'True Income', 'Predicted_Income': 'Predicted Income'}
)
fig_train.show()

### Visualize Testing Results

In [48]:
fig_test = px.scatter(
    TestingResults,
    x='True_Income',
    y='Predicted_Income',
    title='True vs. Predicted Income (Testing Data)',
    labels={'True_Income': 'True Income', 'Predicted_Income': 'Predicted Income'}
)
fig_test.show()